# 3. Deep Hedging Model

In this notebook, we implement and train a neural network that learns to hedge a European call option dynamically using simulated market data.

> **Goal.** Train a trading policy $(a_t)$ that minimizes **tail risk** (CVaR) of the terminal hedging error
> $$
X \;=\; -Z_T \;+\; \sum_{t=0}^{T-1} a_t\,\Delta S_t \;-\; \underbrace{\gamma\sum_{t=0}^{T-1} S_t\,|a_t - a_{t-1}|}_{\text{transaction cost}}
 $$
> on simulated paths from Notebook 2. We do this end-to-end with a recurrent policy network, a symbolic rollout, and a Rockafellar–Uryasev (RU) CVaR head.

---

## 3.1) What this notebook consumes / produces

**Inputs (from Notebook 2):**
- `data/S_train.npy`, `S_val.npy`, `S_test.npy`  — each shape `(N, n_steps+1)`
- `data/Z_T_train.npy`, `Z_T_val.npy`, `Z_T_test.npy`  — each shape `(N,)`
- `data/meta.json` — `S0`, `K`, `T`, `n_steps`, `dt`, plus knobs like `alpha`, `gamma`, `h_max` (optional)

**Outputs (used by Notebook 4):**
- Trained weights (best by tail metrics)
- Arrays for backtesting: $(X)$, $(\text{PnL})$, $(\text{cost})$, $(\text{turnover})$, $(Z_T)$, and optionally positions $(a_t)$.

---

## 3.2) Setup & reproducibility

**What to do here:**
- Import `numpy`, `tensorflow` (Keras), `json`, `pathlib`.
- Fix seeds (`tf.keras.utils.set_random_seed(SEED)`).
- Read `meta.json`, unpack: `S0, K, T_years, n_steps, dt`.  
- Choose training knobs (typical defaults shown; adjust later in §12):
  - `ALPHA = 0.90` (CVaR level)
  - `GAMMA = 0.001` (turnover cost scale; ≈10 bps)
  - `H_MAX = 5` (position clamp \(|a_t|\le H_{\max}\))
  - `BATCH`, `EPOCHS`, `LR`
- Load arrays `S_*` and `Z_T_*` and print shapes (sanity).

In [1]:
# ============================================================
# Notebook 3 (Keras) — Deep Hedging with CVaR Utility
# ============================================================
# Requirements: TensorFlow 2.x (tf.keras), numpy, Keras 3
# Inputs:   data/S_train.npy, S_val.npy, S_test.npy
#           data/Z_T_train.npy, Z_T_val.npy, Z_T_test.npy
#           data/meta.json  (S0, K, T, n_steps, dt, alpha, etc.)
# Output:   results/hedging_eval_test_keras.npz  (keys: V_T, Z_T)
# ============================================================

import os, json, math, time
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"  # quiet TF logs

from pathlib import Path
import numpy as np
import tensorflow as tf
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
# --------------------------
# 0) Repro + folders
# --------------------------
SEED = 42
tf.keras.utils.set_random_seed(SEED)

DATA_DIR = Path("data")
RESULTS_DIR = Path("results"); RESULTS_DIR.mkdir(exist_ok=True)

# --------------------------
# 1) Load meta + raw arrays
# --------------------------
with open(DATA_DIR/"meta_x.json", "r") as f:
    meta = json.load(f)

S0       = float(meta["S0"])
K        = float(meta["K"])
T_years  = float(meta["T"])
n_steps  = int(meta["n_steps"])
dt       = float(meta["dt"])

# --------------------------
# 2) Training knobs
# --------------------------
ALPHA  = 0.9 # CVaR tail level
GAMMA  = 0.0005 # Transaction cost scale γ = 0.001 (10 bps per unit turnover)
H_MAX  = 3 # max abs(position)
BATCH  = 400 # for ALPHA=0.95 ~ 400
EPOCHS = 150
LR     = 1e-4

opt_train = np.load(DATA_DIR/"opt_train_vol.npy")
opt_val   = np.load(DATA_DIR/"opt_val_vol.npy")
opt_test  = np.load(DATA_DIR/"opt_test_vol.npy")

S_train = np.load(DATA_DIR/"S_train_vol.npy")   # (N_train, n_steps+1)
S_val   = np.load(DATA_DIR/"S_val_vol.npy")
S_test  = np.load(DATA_DIR/"S_test_vol.npy")

K_train = np.load(DATA_DIR/"K_train_vol.npy")   # (N_train,)
K_val   = np.load(DATA_DIR/"K_val_vol.npy")
K_test  = np.load(DATA_DIR/"K_test_vol.npy")

Z_T_train = np.load(DATA_DIR/"Z_T_train_vol.npy").astype(np.float32)
Z_T_val   = np.load(DATA_DIR/"Z_T_val_vol.npy").astype(np.float32)
Z_T_test  = np.load(DATA_DIR/"Z_T_test_vol.npy").astype(np.float32)

print(f"Loaded: S_train {S_train.shape}, S_val {S_val.shape}, S_test {S_test.shape}")
print(f"Payoffs: Z_T_train {Z_T_train.shape}, Z_T_val {Z_T_val.shape}, Z_T_test {Z_T_test.shape}")
print(f"Strike Prices: K_train {K_train.shape}, K_val {K_val.shape}, K_test {K_test.shape}")
print(f"n_steps = {n_steps}, N_train = {S_train[:,0].shape}, N_val = {S_val[:,0].shape}, N_test = {S_test[:,0].shape}")

Loaded: S_train (8000, 253), S_val (5000, 253), S_test (5000, 253)
Payoffs: Z_T_train (8000,), Z_T_val (5000,), Z_T_test (5000,)
Strike Prices: K_train (8000,), K_val (5000,), K_test (5000,)
n_steps = 252, N_train = (8000,), N_val = (5000,), N_test = (5000,)


## 3.3) Build *causal* features and $(\Delta S)$

**What to do here:**
- From each `S` (shape `(N, n_steps+1)`) compute:
  - $(S_t := S[:, 0:n\_steps])$
  - $(S_{t+1} := S[:, 1:n\_steps+1])$
  - $(\Delta S_t = S_{t+1} - S_t)$ → shape `(N, n_steps, 1)`
- Causal features per time step $(t)$ (shape `(N, n_steps, 6)`):
  1. `price_norm`: $(S_t / S_0)$
  2. `moneyness`: $(K_t / S_0)$
  3. `tau`: linear grid $(\tau_t = (T - t)/T)$
  4. `logR_past`: **lagged** log return; at $(t=0)$ set to `0`, for $(t>0)$ use $(\log S_t - \log S_{t-1})$
  5. `vol_ann`: rolling std of `logR_past` over a window (e.g., 20), annualized by $(\sqrt{1/dt})$
  6. `normalized Delta`: $\Delta$
  7. `normalized Gamma`: $\Gamma$
  8. `normalized Vega`: $Vega$
  9. `option_type`: Call/Put, binary values 0 or 1.

In [2]:
# --------------------------
# 3) Causal features: [price_norm, moneyness, tau, log_return, realized_vol_annualized, norm_delta, norm_gamma, norm_vega, call/put]
# --------------------------
def bs_delta(St, K, tau, vol_ann, opt_type, r=0.0):
    """
    St: (N, n_steps)
    K:  (N,) (strike per path)
    tau, vol_ann: (N, n_steps)
    opt_type: (N,)  0=call, 1=put
    returns delta: (N, n_steps)
    """
    K = np.asarray(K).reshape(-1)
    Kt = K[:, None]   # broadcast
    
    eps = 1e-6
    tau_safe = np.clip(tau, eps, None)
    vol_safe = np.clip(vol_ann, eps, None)

    # avoid divide-by-zero where tau == 0
    with np.errstate(divide='ignore', invalid='ignore'):
        d1 = np.where(
            tau_safe > 0,
            (np.log(St / Kt) + (r + 0.5 * vol_safe**2) * tau_safe) / (vol_safe * np.sqrt(tau_safe)),
            0.0
        )

    delta = np.where(opt_type[:, None] == 0, norm.cdf(d1), norm.cdf(d1) - 1.0)
    return delta

def bs_d1_phi(St, K, tau, vol_ann, r=0.0, eps=1e-6):
    """
    St: (N, n_steps)   - spot per path/time (not including final step)
    K:  (N,)           - strike per path (constant)
    tau, vol_ann: (N, n_steps)
    returns: d1, phi, vol_safe, tau_safe  (all shape (N, n_steps))
    """
    K = np.asarray(K).reshape(-1)            # ensure 1d
    Kt = K[:, None]                          # broadcast to (N, n_steps)

    tau_safe = np.clip(tau, eps, None)
    vol_safe = np.clip(vol_ann, eps, None)

    d1 = (np.log(St / Kt) + (r + 0.5 * vol_safe**2) * tau_safe) / (vol_safe * np.sqrt(tau_safe))
    d1 = np.clip(d1, -10.0, 10.0)
    phi = norm.pdf(d1)
    return d1, phi, vol_safe, tau_safe


# ---------- Features builder (robust to K being array or matrix) ----------
def build_features_from_S(S, K, S0, n_steps, dt, win=20):
    """
    S: (N, n_steps+1)
    K: (N,) or (N, n_steps+1). This function handles either but will
       prefer treating K as per-path strike if 1D.
    Returns: dSt, dK, X, logR_past, vol_ann
    """
    S = np.asarray(S)
    K = np.asarray(K)

    N = S.shape[0]
    St = S[:, :-1]        # (N, n_steps)
    St_p1 = S[:, 1:]

    # dS shape (N, n_steps, 1)
    dSt = (St_p1 - St)[:, :, None].astype(np.float32)

    price_norm = St / S0

    # Standardize strike shape:
    if K.ndim == 1:
        # strike per path (constant through time)
        Kt = np.broadcast_to(K[:, None], St.shape)  # (N, n_steps)
        # dK is zero if strike is constant
        dK = np.zeros_like(dSt, dtype=np.float32)
    elif K.ndim == 2:
        # strike matrix: use all but last column to align with St
        if K.shape[1] != St.shape[1] + 1:
            raise ValueError(f"K matrix must have n_steps+1 columns to align with S. got {K.shape}")
        Kt = K[:, :-1]
        dK = (K[:, 1:] - K[:, :-1])[:, :, None].astype(np.float32)
    else:
        raise ValueError(f"Unexpected K ndim: {K.ndim}")

    # Avoid zero division early:
    Kt = Kt.astype(np.float32)

    moneyness = (St - Kt) / (Kt + 1e-12)

    # log returns (for vol feature)
    r_next = np.diff(np.log(S), axis=1).astype(np.float32)
    logR_past = np.zeros_like(r_next, dtype=np.float32)
    if n_steps > 1:
        logR_past[:, 1:] = r_next[:, :-1]

    # tau vector (normalized time remaining)
    tau_vec = np.linspace(1.0 - 1.0/n_steps, 0.0, n_steps, dtype=np.float32)
    tau = np.broadcast_to(tau_vec, (N, n_steps)).astype(np.float32)

    # realized vol (rolling past window)
    vol = np.empty_like(logR_past, dtype=np.float32)
    for t in range(n_steps):
        l = max(0, t - win + 1)
        w = logR_past[:, l:t+1]
        vol[:, t] = w.std(axis=1)
    steps_per_year = int(round(1.0 / dt))
    vol_ann = vol * np.sqrt(steps_per_year)

    # Greeks using broadcast-friendly functions
    d1, phi, vol_safe, tau_safe = bs_d1_phi(St, K if K.ndim == 1 else Kt, tau, vol_ann, r=0.0, eps=1e-6)
    gamma = phi / (St * vol_safe * np.sqrt(tau_safe) + 1e-12)
    vega  = St * phi * np.sqrt(tau_safe)
    gamma = np.clip(gamma, -1e3, 1e3)
    vega  = np.clip(vega, -1e3, 1e3)

    X = np.stack([price_norm, moneyness, tau, logR_past, vol_ann, gamma, vega], axis=-1).astype(np.float32)

    return dSt, dK, X, logR_past.astype(np.float32), vol_ann.astype(np.float32)
    
dSt_train, dK_train, X_train, logR_train, vol_train = build_features_from_S(S_train, K_train, S0, n_steps, dt, win=20)
dSt_val, dK_val, X_val,   logR_val,   vol_val   = build_features_from_S(S_val, K_val,   S0, n_steps, dt, win=20)
dSt_test, dK_test, X_test,  logR_test,  vol_test  = build_features_from_S(S_test, K_test,  S0, n_steps, dt, win=20)

Z_T_train = np.asarray(Z_T_train, dtype=np.float32).reshape(-1)
Z_T_val   = np.asarray(Z_T_val,   dtype=np.float32).reshape(-1)
Z_T_test  = np.asarray(Z_T_test,  dtype=np.float32).reshape(-1)

# Use the raw S arrays (they exist in your notebook)
St_train = S_train[:, :-1]   # shape (N, n_steps)
St_val   = S_val[:, :-1]
St_test  = S_test[:, :-1]

delta_train = bs_delta(St_train, K_train, X_train[:, :, 2], X_train[:, :, 4], opt_train, r=0.0)
delta_val   = bs_delta(St_val,   K_val,   X_val[:, :, 2],   X_val[:, :, 4],   opt_val,   r=0.0)
delta_test  = bs_delta(St_test,  K_test,  X_test[:, :, 2],  X_test[:, :, 4],  opt_test,  r=0.0)


# ---- Broadcast Delta along last dim to make it (N, n_steps, 1) like your other features ----
delta_train_feat = delta_train[:,:,None].astype(np.float32)
delta_val_feat   = delta_val[:,:,None].astype(np.float32)
delta_test_feat  = delta_test[:,:,None].astype(np.float32)

# ---- NEW option type ----
opt_train = np.load(DATA_DIR/"opt_train_x.npy")  # shape (N_train,)
opt_val   = np.load(DATA_DIR/"opt_val_x.npy")
opt_test  = np.load(DATA_DIR/"opt_test_x.npy")

# Broadcast each option type along the time axis so it’s (N, n_steps, 1)
opt_train_feat = np.tile(opt_train[:, None, None], (1, X_train.shape[1], 1))
opt_val_feat   = np.tile(opt_val[:, None, None],   (1, X_val.shape[1], 1))
opt_test_feat  = np.tile(opt_test[:, None, None],  (1, X_test.shape[1], 1))

# ---- Concatenate to existing features ----
X_train = np.concatenate([X_train, delta_train_feat], axis=-1)
X_val   = np.concatenate([X_val,   delta_val_feat],   axis=-1)
X_test  = np.concatenate([X_test,  delta_test_feat],  axis=-1)

# Extract Greeks correctly: (N, n_steps, 3)
greeks_train = X_train[:, :, -3:]
greeks_val   = X_val[:, :, -3:]
greeks_test  = X_test[:, :, -3:]

# Flatten to (N*n_steps, 3) for sklearn
greeks_train_flat = greeks_train.reshape(-1, 3)
greeks_val_flat   = greeks_val.reshape(-1, 3)
greeks_test_flat  = greeks_test.reshape(-1, 3)

# Normalize only greeks
scaler_greeks = StandardScaler()
greeks_train_norm = scaler_greeks.fit_transform(greeks_train_flat)
greeks_val_norm   = scaler_greeks.transform(greeks_val_flat)
greeks_test_norm  = scaler_greeks.transform(greeks_test_flat)

# Reshape back to (N, n_steps, 3)
X_train[:, :, -3:] = greeks_train_norm.reshape(X_train.shape[0], X_train.shape[1], 3)
X_val[:, :, -3:]   = greeks_val_norm.reshape(X_val.shape[0], X_val.shape[1], 3)
X_test[:, :, -3:]  = greeks_test_norm.reshape(X_test.shape[0], X_test.shape[1], 3)

# Concatenate with the 4 existing features → now we have 5
X_train = np.concatenate([X_train, opt_train_feat], axis=-1)
X_val   = np.concatenate([X_val,   opt_val_feat],   axis=-1)
X_test  = np.concatenate([X_test,  opt_test_feat],  axis=-1)


print("Feature tensor shapes:",
      "\n dSt_train:", dSt_train.shape,
      "\n  X_train:", X_train.shape,
      "\n  X_val:  ", X_val.shape,
      "\n  X_test: ", X_test.shape,
      "\n  K_train:", K_train.shape,
      "\n  K_val:  ", K_val.shape,
      "\n  K_test: ", K_test.shape,
      "\n  Z_train:", Z_T_train.shape,
      "\n  Z_val:  ", Z_T_val.shape,
      "\n  Z_test: ", Z_T_test.shape)

Feature tensor shapes: 
 dSt_train: (8000, 252, 1) 
  X_train: (8000, 252, 9) 
  X_val:   (5000, 252, 9) 
  X_test:  (5000, 252, 9) 
  K_train: (8000,) 
  K_val:   (5000,) 
  K_test:  (5000,) 
  Z_train: (8000,) 
  Z_val:   (5000,) 
  Z_test:  (5000,)


In [3]:
n_feat   = X_train.shape[-1] 
n_train  = X_train.shape[0]
n_val  = X_val.shape[0]
n_test  = X_test.shape[0]

## 3.4) Build `tf.data` pipelines

**What to do here:**
- Construct dictionaries per split:
  - `inputs = {"features": X, "dS": dS, "Z_T": Z_T}`  
    (keep `Z_T` **1D** `(N,)`
  - `labels = zeros(N)`  (dummy; our loss reads from the graph)
- Create datasets:
  - `train_ds = Dataset.from_tensor_slices((inputs, labels)).cache().shuffle(...).batch(BATCH).prefetch(AUTOTUNE)`
  - Similarly for `val_ds` (no shuffle) and `test_ds`.
- **Peek a batch** and print dtypes/shapes to confirm:
  - `features: (BATCH, n_steps, 4)`
  - `dS:       (BATCH, n_steps, 1)`
  - `Z_T:      (BATCH,)`

In [4]:
# --------------------------
# 4) Datasets
# --------------------------

inputs = {
    "features": X_train[:,:,:],
    "dS": dSt_train[:,:,:],
    "Z_T": Z_T_train[:]
}

val_inputs = {
    "features": X_val[:,:,:],
    "dS": dSt_val[:,:,:],
    "Z_T": Z_T_val[:]
}

test_inputs = {
    "features": X_test[:,:,:],
    "dS": dSt_test[:,:,:],
    "Z_T": Z_T_test[:]
}

input_label = np.zeros((n_train),dtype=np.float32)
val_label = np.zeros((n_val),dtype=np.float32)
test_label = np.zeros((n_test),dtype=np.float32)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = (tf.data.Dataset.from_tensor_slices((inputs, input_label))
            .cache().shuffle(8192, seed=SEED, reshuffle_each_iteration=True)
            .batch(BATCH).prefetch(AUTOTUNE))

val_ds   = (tf.data.Dataset.from_tensor_slices((val_inputs, val_label))
            .cache().batch(BATCH).prefetch(AUTOTUNE))

test_ds  = (tf.data.Dataset.from_tensor_slices((test_inputs, test_label))
            .batch(BATCH).prefetch(AUTOTUNE))

## 3.5) Zero-hedge baseline (validation)

**What to do here:**
- Baseline wealth if you **do not trade**: $(X^{(0)} = -Z_T)$.
- Compute on **all** validation paths:
  - `mean_baseline = mean(X0)`
  - `VaR_α` and `CVaR_α` at `ALPHA` (note: sort ascending, take worst tail)

---

In [5]:
# --------------------------
# 5) Baseline 
# --------------------------
wealth = - Z_T_val

val_mean_baseline  = wealth.mean()
sorted_wealth = np.sort(wealth)

k = int(np.ceil((1 - ALPHA) * sorted_wealth.size))
values = sorted_wealth[:k]

val_cvar_baseline = values.mean()
val_p05_baseline = np.percentile(wealth, 5)

print(f"val_cvar_baseline: {val_cvar_baseline:.6f}")
print(f"val_mean_baseline: {val_mean_baseline:.6f}")
print(f"val_p05_baseline: {val_p05_baseline:.6f}")

val_cvar_baseline: -105.294373
val_mean_baseline: -22.909412
val_p05_baseline: -88.195668


## 3.6) Policy network (backbone)

**What to do here:**
- Define three **Inputs**:
  - `features`: `(n_steps, 4)` — the only tensor used by the policy
  - `dS`: `(n_steps, 1)` — used *later* in rollout, not inside the policy
  - `Z_T`: `()` — scalar terminal payoff, used *later* in rollout, not inside the policy
- Build a small recurrent backbone, e.g. GRU→GRU→Dense:
  - GRU(32, return_sequences=True) → GRU(16, return_sequences=True) → Dense(32, relu)
  - Final Dense to 1 → raw position `a_raw(t)`
  - **Clamp**: `a(t) = H_MAX * tanh(a_raw(t))` to bound $(|a_t|\le H_{\max})$

**Why these choices:**
- GRUs capture temporal context without heavy complexity.
- `tanh` clamp stabilizes training, enforces realistic position bounds.
- `dS` and `Z_T` must *not* leak into the policy — they are used only in P&L computation.

---

In [6]:
from tensorflow import keras
from tensorflow.keras import layers

In [7]:
# --------------------------
# 6) Model
# --------------------------

features_in = keras.Input(shape=(n_steps, n_feat), name="features")
dS_in = keras.Input(shape=(n_steps,1), name="dS")
Z_T_in = keras.Input(shape=(), name="Z_T")

H = layers.GRU(32, return_sequences=True, name="gru_32")(features_in)
H = layers.GRU(16, return_sequences=True, name="gru_16")(H)
H = layers.Dense(32, activation="relu", name="dense_32")(H)

a_raw = layers.Dense(1, name="a_raw")(H)
a = layers.Lambda(lambda z: tf.cast(H_MAX, z.dtype) * tf.tanh(z),name="a")(a_raw)

policy = keras.Model(inputs=[features_in, dS_in, Z_T_in], outputs=a, name="policy_backbone")
policy.summary()

Model: "policy_backbone"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 features (InputLayer)       [(None, 252, 9)]             0         []                            
                                                                                                  
 gru_32 (GRU)                (None, 252, 32)              4128      ['features[0][0]']            
                                                                                                  
 gru_16 (GRU)                (None, 252, 16)              2400      ['gru_32[0][0]']              
                                                                                                  
 dense_32 (Dense)            (None, 252, 32)              544       ['gru_16[0][0]']              
                                                                                    

## 3.7) Symbolic rollout (within the Keras graph)

**What to do here (all tensor ops, no Python loops):**
1. **Positions:** `a = policy(features)` → shape `(B, n_steps, 1)`
2. **Turnover:** `a_prev` = shift of `a` with a zero at $(t=0)$;  
   `turnover = |a - a_prev|`
3. **Transaction cost:** get $(S_t)$ from features: `S_t = features[:,:,0] * S0` (then `[..., None]` to match dims);  
   `cost = GAMMA * sum_t (S_t * turnover)` → shape `(B, 1)` then squeeze to `(B,)`
4. **PnL:** `PnL = sum_t (a * dS)` → `(B,)`
5. **Terminal wealth:** `X = -Z_T + PnL - cost` → `(B,)`

**Monitoring tensors (for `add_metric`):**
- `mean_X = mean(X)`
- `mean_PnL = mean(PnL)`
- `mean_cost = mean(cost)`
- `turnover_mean = mean(turnover)` (averaged over batch & time)
- `bound_frac = mean( I[|a| >= 0.95*H_MAX] )`
- **Empirical tail stats on the batch** (for logging only):
  - $(k = \lceil (1-\alpha)$,$\text{batch\_size} \rceil)$
  - `X_sorted = sort(X)` (ascending)
  - `var_emp = X_sorted[k-1]`, `cvar_emp = mean(X_sorted[:k])`

In [8]:
# --------------------------
# 7) Rollout
# --------------------------

a_prev = tf.concat([tf.zeros_like(a[:, :1, :]), a[:, :-1, :]], axis=1)
turnover = tf.abs(a - a_prev)

S_t = features_in[:, :, 0] * tf.cast(S0, tf.float32)
S_t = tf.expand_dims(S_t, axis=-1)

cost = GAMMA * tf.reduce_sum(S_t * turnover, axis=[1,2])
PnL  = tf.reduce_sum(a * dS_in, axis=[1,2])

X = -Z_T_in + PnL - cost  # shape: (batch,)

mean_X = tf.reduce_mean(X)
mean_PnL = tf.reduce_mean(PnL)
mean_cost = tf.reduce_mean(cost)
turnover_mu = tf.reduce_mean(turnover)
bound_frac  = tf.reduce_mean(tf.cast(tf.abs(a) >= 0.95*H_MAX, tf.float32))

k = tf.cast(tf.math.ceil((1.0 - ALPHA) * tf.cast(tf.shape(X)[0], tf.float32)), tf.int32)
X_sorted = tf.sort(X)
var_emp = X_sorted[k-1]
cvar_emp = tf.reduce_mean(X_sorted[:k])

## 3.8) RU CVaR head (loss layer)

**What to do here:**
- Add a custom layer **RUHead(α)** with a **trainable** scalar $(\tau)$ (initialized e.g. near the baseline VaR or a constant):
  $$
  \ell(X;\tau) \;=\; \frac{(\tau - X)^+}{1-\alpha} \;-\; \tau
  $$
- Properties:
  - Minimizing $( \mathbb{E}[\ell(X;\tau)] )$ over $(\tau)$ yields $(\tau = \text{VaR}_\alpha(X))$.
  - The minimized value equals $( \text{CVaR}_\alpha(X) )$.
- Pipe the rollout wealth \(X\) into RUHead to get per-sample losses $(\ell)$, then average.

**Optional auxiliary penalty (keep small):**
- $(\beta \cdot \text{mean}(|X|))$ to nudge overall MAE down, but **do not** let it dominate.

In [9]:
# --------------------------
# 8) CVaR head 
# --------------------------

BETA = 0.01

class RUHead(tf.keras.layers.Layer):
    def __init__(self, alpha,tau_init=-66.0,  **kwargs):
        super().__init__(**kwargs)
        self.alpha = float(alpha)
        self.tau_init = float(tau_init)

    def build(self, input_shape):
        self.tau = self.add_weight(
            name="tau",
            shape=(),
            initializer=tf.keras.initializers.Constant(self.tau_init),
            trainable=True,
            dtype=tf.float32
        )
    
    def call(self, X):
        X = tf.cast(X, self.tau.dtype)

        if X.shape.rank == 2 and X.shape[-1] == 1:
            X = tf.squeeze(X, axis=-1)
            
        hinge = tf.nn.relu(self.tau - X)
        ell = hinge / (1.0 - self.alpha) - self.tau
        self.add_metric(tf.identity(self.tau), name="tau", aggregation="mean")
        return ell


def identify_loss(y_true, y_pred):
    return y_pred

ru = RUHead(alpha=ALPHA, name="ru_head")
ell = ru(X)
mae_penalty = BETA * tf.reduce_mean(tf.abs(X))   

## 3.9) Compose the training model

**What to do here:**
- Create `loss_model = Model([features_in, dS_in, Z_T_in] → RUHead(X))`.
- Attach `add_metric(...)` for the monitors from §6 (mean_X, mean_PnL, mean_cost, turnover_mean, bound_frac, var_emp, cvar_emp) and also expose `tau` from RUHead.
- **Compile** with Adam (`learning_rate = LR`, optional `clipnorm=1.0`).
- Use an **identity loss** (`return y_pred`) because the RUHead already outputs $(\ell)$.


In [10]:
# --------------------------
# 9) Loss Model
# --------------------------

loss_model = keras.Model(inputs=[features_in, dS_in, Z_T_in],
                         outputs=ell,
                         name="loss")

loss_model.add_loss(mae_penalty)

loss_model.add_metric(mean_X, name="mean_X", aggregation="mean")
loss_model.add_metric(mean_PnL, name="mean_PnL", aggregation="mean")
loss_model.add_metric(mean_cost, name="mean_Cost", aggregation="mean")
loss_model.add_metric(turnover_mu, name="turnover_mean", aggregation="mean")
loss_model.add_metric(bound_frac, name="bound_frac", aggregation="mean")
loss_model.add_metric(cvar_emp, name="cvar_emp", aggregation="mean")
loss_model.add_metric(var_emp, name="var_emp", aggregation="mean")

mean_abs_X = tf.reduce_mean(tf.abs(X))
loss_model.add_metric(mean_abs_X, name="mean_abs_X", aggregation="mean")

loss_model.compile(optimizer=keras.optimizers.Adam(learning_rate = LR,
                                                   clipnorm=1.0),
                                                   loss=identify_loss,
                                                   )    

## 3.10) Training loop & callbacks

**What to do here:**
- Set callbacks:
  - `ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-5)`
  - `EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)`
  - `ModelCheckpoint(..., monitor="val_cvar_emp", mode="min")` to keep best tails
  - `TerminateOnNaN()`
- `fit(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=cbs)`
- **How to read logs:**
  - `loss / val_loss` ≈ batch mean of RU loss (proxy for CVaR)
  - `tau` should settle near empirical VaR
  - `cvar_emp` (val) should **decrease** vs baseline from §4
  - `mean_cost`, `turnover_mean` give realism signals
  - `bound_frac` near zero (not slamming into the clamp)

In [11]:
# --------------------------
# 10) Training
# --------------------------

cbs = [
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", mode="min",
        factor=0.5, patience=5, min_lr=1e-5, verbose=1
    ),
    keras.callbacks.EarlyStopping(
        monitor="val_loss", mode="min",
        patience=12, restore_best_weights=True, min_delta=1e-3, verbose=1
    ),
    # save weights for the best tail (CVaR)
    keras.callbacks.ModelCheckpoint(
        filepath=str(RESULTS_DIR / "best_tail_by_cvar.weights_x.h5"),
        monitor="val_cvar_emp", mode="max",
        save_best_only=True, save_weights_only=True, verbose=1
    ),
    # optional: also save weights for best VaR (tau)
    keras.callbacks.ModelCheckpoint(
        filepath=str(RESULTS_DIR / "best_tail_by_var.weights_x.h5"),
        monitor="val_tau", mode="max",
        save_best_only=True, save_weights_only=True, verbose=1
    ),
    keras.callbacks.ModelCheckpoint(
        filepath=str(RESULTS_DIR / "best_center_by_absX.weights_x.h5"),
        monitor="val_mean_abs_X", mode="min",
        save_best_only=True, save_weights_only=True, verbose=1
    ),
    keras.callbacks.TerminateOnNaN(),
]

history = loss_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=cbs,
    verbose=1
)

Epoch 1/150
20/20 [==============================] - ETA: 0s - loss: 89.9008 - tau: -66.0004 - mean_X: -22.7944 - mean_PnL: 0.3775 - mean_Cost: 0.2181 - turnover_mean: 0.0171 - bound_frac: 0.0000e+00 - cvar_emp: -89.2121 - var_emp: -65.7280 - mean_abs_X: 30.8431
Epoch 1: val_cvar_emp improved from -inf to -81.07971, saving model to results\best_tail_by_cvar.weights_x.h5

Epoch 1: val_tau improved from -inf to -66.00048, saving model to results\best_tail_by_var.weights_x.h5

Epoch 1: val_mean_abs_X improved from inf to 28.89066, saving model to results\best_center_by_absX.weights_x.h5
20/20 [==============================] - 9s 334ms/step - loss: 89.9008 - tau: -66.0004 - mean_X: -22.7944 - mean_PnL: 0.3775 - mean_Cost: 0.2181 - turnover_mean: 0.0171 - bound_frac: 0.0000e+00 - cvar_emp: -89.2121 - var_emp: -65.7280 - mean_abs_X: 30.8431 - val_loss: 82.3170 - val_tau: -66.0005 - val_mean_X: -22.2767 - val_mean_PnL: 0.6191 - val_mean_Cost: 0.2125 - val_turnover_mean: 0.0166 - val_bound_fr

## 3.11) Global validation snapshot

**What to do here:**
- Build a lightweight `eval_model` that outputs **[X, cost, turnover, PnL]** given inputs (reuses the trained backbone + rollout).
- Run it on **all** validation paths (large batch; no shuffle).
- Compute and print:
  - `mean(X)` vs baseline mean of `-Z_T`
  - `VaR_α(X)` and `CVaR_α(X)` vs baseline
  - `mean(cost)` and **average total turnover per path** (sum over time, average over paths)
- Optionally store arrays for Notebook 4 plots.

In [12]:
# --------------------------
# 11) Global Validation
# --------------------------

# Collect full validation set (inputs + targets)
xb_full = {"features": [], "dS": [], "Z_T": []}
yb_full = []

for xb, yb in val_ds:   # iterate batches
    for k in xb_full: 
        xb_full[k].append(xb[k].numpy())
    yb_full.append(yb.numpy())

# Concatenate into single arrays
for k in xb_full:
    xb_full[k] = np.concatenate(xb_full[k], axis=0)
yb_full = np.concatenate(yb_full, axis=0)


eval_model = keras.Model(
    inputs=[features_in, dS_in, Z_T_in],
    outputs=[X, cost, turnover, PnL]
)

X_val, cost_val, turnover_val, PnL_val = eval_model.predict(
    [xb_full["features"], xb_full["dS"], xb_full["Z_T"]],
    batch_size=1024, 
    verbose=1
)

N = X_val.shape[0]

# Mean
mean_XG = np.mean(X_val)

# VaR at 90% (10th percentile)
k = int(np.ceil((1 - ALPHA) * N))
X_sorted = np.sort(X_val)
VaR_90 = X_sorted[k-1]

# CVaR at 90% (average of worst 10%)
CVaR_90 = np.mean(X_sorted[:k])

# Mean cost
mean_cost = np.mean(cost_val)

# Total turnover per path (sum across timesteps, then average over paths)
turnover_total = np.mean(np.sum(turnover_val, axis=1))

X_zero = -xb_full["Z_T"]  # wealth is just -Z_T

# Zero-hedge baseline
X0_sorted = np.sort(X_zero)
mean_baseline = np.mean(X_zero)


var_baseline = X0_sorted[k-1]

hedged_mae = np.mean(np.abs(X_val))             # from eval_model outputs
baseline_mae = np.mean(np.abs(-xb_full["Z_T"])) # zero-hedge
print(f"MAE: model {hedged_mae:.3f} vs baseline {baseline_mae:.3f}")

print("=== Global Validation Evaluation ===")
print(f"Mean(XG): {mean_XG:.2f}  vs baseline {mean_baseline}")
print(f"VaR_90 : {VaR_90:.2f}  vs baseline {var_baseline:.2f}")
print(f"CVaR_90: {CVaR_90:.2f}  vs baseline {val_cvar_baseline:.2f}")
print(f"Mean Cost: {mean_cost:.4f}")
print(f"Avg Turnover: {turnover_total:.4f}")

5/5 [==============================] - 1s 128ms/step
MAE: model 23.882 vs baseline 22.909
=== Global Validation Evaluation ===
Mean(XG): -22.65  vs baseline -22.909412384033203
VaR_90 : -37.01  vs baseline -67.86
CVaR_90: -49.66  vs baseline -105.29
Mean Cost: 0.2044
Avg Turnover: 4.0215


## 3.12) Test set evaluation & saving artifacts

**What to do here:**
- Repeat §11 on `test_ds` and report the same metrics.
- **Save** to `results/hedging_eval_test_keras.npz`, e.g. keys:
  - `X` (terminal wealth), `Z_T`, `PnL`, `cost`, `turnover`, and possibly positions `a`
- These feed Notebook 4 for richer diagnostics/plots.

In [13]:
# --------------------------
# 12) Testing
# --------------------------

xb_full = {"features": [], "dS": [], "Z_T": []}
yb_full = []

for xb, yb in test_ds:   # iterate batches
    for k in xb_full: 
        xb_full[k].append(xb[k].numpy())
    yb_full.append(yb.numpy())

for k in xb_full:
    xb_full[k] = np.concatenate(xb_full[k], axis=0)
yb_full = np.concatenate(yb_full, axis=0)

X_test, cost_test, turnover_test, PnL_test = eval_model.predict(
    [xb_full["features"], xb_full["dS"], xb_full["Z_T"]],
    batch_size=1024,
    verbose=1
)

N = X_test.shape[0]

mean_XG = np.mean(X_test)

k = int(np.ceil((1 - ALPHA) * N))
X_sorted = np.sort(X_test)
VaR_90 = X_sorted[k-1]
CVaR_90 = np.mean(X_sorted[:k])

mean_cost = np.mean(cost_test)
turnover_total = np.mean(np.sum(turnover_test, axis=1))

print("=== Global Test Evaluation ===")
print(f"Mean(X): {mean_XG:.2f}  vs baseline {mean_baseline:.2f}")
print(f"VaR_90 : {VaR_90:.2f}  vs baseline {var_baseline:.2f}")
print(f"CVaR_90: {CVaR_90:.2f}  vs baseline {val_cvar_baseline:.2f}")
print(f"Mean Cost: {mean_cost:.4f}")
print(f"Avg Turnover: {turnover_total:.4f}")

5/5 [==============================] - 1s 125ms/step
=== Global Test Evaluation ===
Mean(X): -23.03  vs baseline -22.91
VaR_90 : -36.57  vs baseline -67.86
CVaR_90: -49.31  vs baseline -105.29
Mean Cost: 0.2018
Avg Turnover: 4.0011


In [14]:
# --------------------------
# 13) SAVE
# --------------------------
# Save V_T (hedged terminal portfolio) and Z_T (payoff) to NPZ
#   X_test, cost_test, turnover_test, PnL_test = eval_model.predict(...)
#   xb_full["Z_T"] from building the test set

V_T_test = (PnL_test.reshape(-1) - cost_test.reshape(-1)).astype(np.float32)  # (N,)
Z_T_test = xb_full["Z_T"].reshape(-1).astype(np.float32)                      # (N,)

# Sanity check
assert V_T_test.shape == Z_T_test.shape, f"Shape mismatch: {V_T_test.shape} vs {Z_T_test.shape}"

out_path = RESULTS_DIR / "hedging_eval_test_keras_xi.npz"
np.savez_compressed(out_path, V_T=V_T_test, Z_T=Z_T_test)
print(f"Saved: {out_path}  (V_T {V_T_test.shape}, Z_T {Z_T_test.shape})")

Saved: results\hedging_eval_test_keras_xi.npz  (V_T (5000,), Z_T (5000,))


In [15]:
# loss_model.load_weights(RESULTS_DIR / "best_tail_by_cvar.weights_x.h5")

## 3.13) Interpreting results

**What good looks like (directionally):**
- `CVaR_α(X)` **much higher** (less negative) than baseline \( \text{CVaR}_\alpha(-Z_T) \)  
  (remember: more negative = worse loss; moving toward zero is improvement)
- `VaR_α(X)` improved similarly
- `mean(X)` may stay negative (you’re minimizing tail, not MSE); small positive is a bonus
- `mean_cost` and `turnover` within realistic bounds
- `bound_frac` small (not exploiting the clamp edge)

**If it looks too good to be true:**
- Check no look-ahead in features (§2)
- Ensure `dS` aligns with time \(t\) features
- Confirm `Z_T` shape is 1D and used only in `X = -Z_T + ...`

---

## 3.14) Hyperparameter guide (what changes what)

- **`ALPHA` (tail level):**  
  Higher (e.g., 0.95) ⇒ focuses on *extreme* worst tail, typically stronger tail protection, possibly larger mean error.  
  Lower (e.g., 0.80) ⇒ softer tail, may improve average metrics but weaker extreme-loss control.
- **`GAMMA` (cost):**  
  Higher ⇒ discourages turnover, reduces cost & noise, might leave some risk unhedged.  
  Lower ⇒ more active hedging, better fit, but higher costs / overfitting risk.
- **`H_MAX` (clamp):**  
  Lower ⇒ policies stay small; safer but under-powered.  
  Higher ⇒ more capacity; watch `bound_frac`.
- **Backbone size (GRU widths):**  
  Larger ⇒ more expressive; use dropout or stronger in case of overfitting.
- **`LR`, `BATCH`:**  
  Tail objectives benefit from moderately large batches (more stable tail estimate). Too small batches produce noisy VaR/CVaR.